<a href="https://colab.research.google.com/github/nikhil-xyz/MNIST_Dataset/blob/main/Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nikhil-xyz/MNIST_Dataset.git

Cloning into 'MNIST_Dataset'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 1), reused 6 (delta 1), pack-reused 0
Unpacking objects: 100% (11/11), done.


In [2]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/MNIST_Dataset/data/fashion-mnist.csv/fashion-mnist_test.csv')

In [11]:
df.shape

(10000, 785)

In [7]:
df.columns

Index(['label', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779', 'pixel780',
       'pixel781', 'pixel782', 'pixel783', 'pixel784'],
      dtype='object', length=785)

In [12]:
train = df['label']

In [14]:
X = df[df.columns[1:]].values
y = df['label']

In [57]:
len(y)

10000

In [21]:
X = X / 255

In [58]:
len(X)

10000

In [59]:
input_shape = X[0].shape

In [60]:
input_shape

(784,)

In [24]:
import tensorflow as tf

In [63]:
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.cache()
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [64]:
train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

In [65]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_shape=input_shape, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='sigmoid'),
])

model.compile(loss='SparseCategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])

In [66]:
model.fit(train, epochs=20, validation_data=val)

Epoch 1/20
437/437 [==============================] - 2s 4ms/step - loss: 0.7710 - accuracy: 0.7177 - val_loss: 0.5547 - val_accuracy: 0.8145
Epoch 2/20
437/437 [==============================] - 1s 3ms/step - loss: 0.5092 - accuracy: 0.8189 - val_loss: 0.5038 - val_accuracy: 0.8270
Epoch 3/20
437/437 [==============================] - 1s 3ms/step - loss: 0.4473 - accuracy: 0.8408 - val_loss: 0.4577 - val_accuracy: 0.8415
Epoch 4/20
437/437 [==============================] - 2s 3ms/step - loss: 0.4082 - accuracy: 0.8543 - val_loss: 0.4460 - val_accuracy: 0.8455
Epoch 5/20
437/437 [==============================] - 2s 3ms/step - loss: 0.3748 - accuracy: 0.8638 - val_loss: 0.4233 - val_accuracy: 0.8455
Epoch 6/20
437/437 [==============================] - 2s 3ms/step - loss: 0.3543 - accuracy: 0.8714 - val_loss: 0.4360 - val_accuracy: 0.8430
Epoch 7/20
437/437 [==============================] - 1s 3ms/step - loss: 0.3351 - accuracy: 0.8769 - val_loss: 0.4202 - val_accuracy: 0.8515
Epoch 

In [78]:
from tensorflow.keras.metrics import CategoricalAccuracy
import numpy as np

In [114]:
from tensorflow.python.ops.gen_array_ops import expand_dims
X_true, y_true = test.as_numpy_iterator().next()
yhat = (model.predict(X_true))
yhat

1/1 [==============================] - 0s 21ms/step


array([[1.00533676e-03, 3.16790210e-05, 3.88944085e-04, 1.06447951e-04,
        3.02926055e-04, 9.99610364e-01, 8.72119889e-03, 3.20793577e-02,
        1.05271407e-03, 1.99028589e-02],
       [3.00545110e-11, 1.00000000e+00, 4.39629321e-06, 1.43336505e-03,
        1.14605682e-05, 1.73190099e-14, 1.19494635e-03, 3.24469699e-13,
        5.73717582e-04, 1.35447443e-11],
       [5.68606119e-05, 2.07634497e-04, 7.94066545e-06, 1.32175777e-02,
        1.23332342e-04, 3.51639897e-01, 6.08938244e-05, 9.98447359e-01,
        1.15190260e-03, 1.39973103e-03],
       [1.64709270e-01, 1.79499213e-04, 9.99254942e-01, 3.00216824e-01,
        3.17371339e-02, 7.54821167e-06, 7.47432590e-01, 2.96452095e-06,
        1.14693178e-03, 3.70811244e-06],
       [1.80125353e-04, 1.08768954e-03, 2.85707472e-04, 3.59687809e-04,
        6.87702978e-03, 2.63146400e-01, 2.80288747e-04, 9.95473742e-01,
        5.12334071e-02, 9.86275434e-01],
       [1.00938860e-05, 1.00000000e+00, 4.53793400e-05, 1.85418352e-02,
   

In [132]:
batch = test.as_numpy_iterator().next()
X_true, y_true = batch
yhat = model.predict(X_true)
hold = [np.argmax(y) for y in yhat]
temp = []
for y in yhat:
  y = np.argmax(y)
  temp.append(y)
# print(temp)
print(y_true)
print(np.array(hold))

1/1 [==============================] - 0s 19ms/step
[5 1 7 2 7 1 7 1 9 2 5 1 2 4 9 0]
[5 1 7 2 7 1 7 1 9 4 5 1 2 1 9 0]


In [133]:
acc = CategoricalAccuracy()
for batch in test.as_numpy_iterator():
  X_true, y_true = batch
  yhat = model.predict(X_true)
  yhat = [np.argmax(y) for y in yhat]
  yhat = np.array(yhat)
  y_true = y_true.flatten()
  yhat = yhat.flatten()
  acc.update_state(y_true, yhat)
print(acc.result().numpy())


1/1 [==============================] - 0s 18ms/step
0.88709676
